In [ ]:
#da fare:
#sistemare report
#sistemare clustering
#classification

In [ ]:
from NewLibraryENGcopia import *

In [ ]:
import glob
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5files/*.h5")

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5filesprova2/*")
for file in list_dir:
    target=1
    stim=0
    if 'healthy' in file:
        target=0
        print('healthy')
    if 'stimulation' in file:
        stim=1
        print('baseline')
    file_name = file.split("/")[-1]
    print(file_name,':','target',target,'stimulation',stim)
    neurons=spike_sorting(file_name,file,1,5,1.5)
    df=poiproc(neurons,target,stim)
    final.append(df)

In [ ]:
concatenated_dataframes = [elem.T for elem in tqdm(final)]
final_result = pd.concat(concatenated_dataframes, axis=0)
dataset=final_result
dataset = dataset.reset_index(drop=True)
dataset

In [ ]:
dataset['Target'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
X = dataset.drop(['Target','Stimulation'],axis=1)
y= dataset['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
cv = StratifiedKFold(4)

In [ ]:
pca = PCA(n_components=5,svd_solver='full')
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.fit(X_train).explained_variance_ratio_,sum(pca.fit(X_train).explained_variance_ratio_))

In [ ]:
params = {'max_depth':np.linspace(2,10,dtype='int'),'criterion':['gini','entropy']}

clf = DecisionTreeClassifier(random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

gs_pca = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)
gs_pca = gs_pca.fit(X_train_pca,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)
print('\n\n Result with PCA\n')
print("Best parameters set found :",gs_pca.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs_pca.predict(X_test_pca)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

In [ ]:
params = {'n_estimators':np.linspace(10,150,dtype='int',num=10),'max_depth':np.linspace(2,5,dtype='int'),'criterion':['gini','entropy']}

clf = RandomForestClassifier(n_jobs=-1,random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

In [ ]:
fs = 10000 #Sampling Frequency
all_data=[]
all_ref=[]
for file in tqdm(list_dir):
    string=file
    data=h5py.File(string,'r')
    data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
    info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
    info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
    labels = info_table['Label']
    readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
    ref=readings[b'Ref']
    len_data=len(readings)
    readings=readings.drop([b'Ref'],axis=1)
    #readings=readings.iloc[0:len_data, 17:23]
    ref=ref[0:len_data]
    all_ref.append(ref)
    print(file,readings.shape)
    all_data.append(readings)

In [ ]:
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filtered_all_data=[]
for file in tqdm(range(len(all_data))):
    prova=all_data[file]
    ref=all_ref[file]
    prova_rows = range(prova.shape[0])
    filt_prova = pd.DataFrame(data = 0, columns=prova.columns, index=prova_rows, dtype = "float32")
    filt_ref=filtfilt(b,a,ref)
    for x in tqdm(range(prova.shape[1])):
        filt_prova.values[:,x] = scipy.signal.filtfilt(b, a, prova.values[:,x])
    for electrode in prova.columns:
        filt_prova[electrode] = filt_prova[electrode] - filt_ref
    filtered_all_data.append(filt_prova)
    filt_prova.shape

# Detection

In [ ]:
pos_ind=[]
neg_ind=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_ind_file=[]
    neg_ind_file=[]
    prova=filtered_all_data[file]
    for electrode in tqdm(prova.columns):
        channel=prova[electrode]
        pos, neg=find_all_spikes(channel)
        pos_ind_file.append(pos)
        neg_ind_file.append(neg)
    pos_ind.append(pos_ind_file)
    neg_ind.append(neg_ind_file)

# Cutouts

In [ ]:
pos_cut=[]
neg_cut=[]
n_pos=[]
n_neg=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=[]
    neg_cut_file=[]
    n_pos_file=[]
    n_neg_file=[]
    prova=filtered_all_data[file]
    pos_ind_file=pos_ind[file]
    neg_ind_file=neg_ind[file]
    for i,electrode in enumerate(tqdm(prova.columns)):
        pos=pos_ind_file[i]
        neg=neg_ind_file[i]
        channel=prova[electrode]
        pos_cut1,n_pos1, neg_cut1,n_neg1 = cut(pos,neg,channel)
        pos_cut_file.append(pos_cut1)
        neg_cut_file.append(neg_cut1)
        n_pos_file.append(n_pos1)
        n_neg_file.append(n_neg1)
    pos_cut.append(pos_cut_file)
    neg_cut.append(neg_cut_file)
    n_pos.append(n_pos_file)
    n_neg.append(n_neg_file)
#savedp = copy.deepcopy(pos_cut)
#savedn = copy.deepcopy(neg_cut)

In [ ]:
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=pos_cut[file]
    neg_cut_file=neg_cut[file]
    for i in (tqdm(range(len(pos_cut_file)))):
        pos_cut_file[i]=mask_cuts(pos_cut_file[i])
        neg_cut_file[i]= mask_cuts(neg_cut_file[i])
    pos_cut[file]=pos_cut_file
    neg_cut[file]=neg_cut_file

# Clustering

In [ ]:
#per canali con più di 30.000 spike, il massimo silhouette è sempre con 2 clusters

In [ ]:
pos_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    #data=filtered_all_data[file]
    #pos_cut_file=pos_cut[file]
    #n_pos_file=n_pos[file]
    final_data_pos=[]
    for channel in (tqdm(range(len(pos_cut_file)))):
        channel_clusters=clus(pos_cut[file][channel],'kmeans',n_pos[file][channel],filtered_all_data[file])
        final_data_pos.append(channel_clusters)
    pos_clusters.append(final_data_pos)

In [ ]:
neg_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    data=filtered_all_data[file]
    neg_cut_file=neg_cut[file]
    n_neg_file=n_neg[file]
    final_data_neg=[]
    for channel in (tqdm(range(len(neg_cut_file)))):
        channel_clusters=clus(neg_cut_file[channel],'kmeans',n_neg_file[channel],data)
        final_data_neg.append(channel_clusters)
    neg_clusters.append(final_data_neg)

In [ ]:
### verificare differenze tra kmeans e fuzzy c means

In [ ]:
neurons=[]
for file in pos_clusters:
    neurons_c=[]
    for channel in file:
        #neurons_c=[]
        for neuron in channel:
            neurons_c.append(neuron)
        #neurons_f.append(neurons_c)
    neurons.append(neurons_c)
len(neurons)

In [ ]:
print(len(pos_clusters))#files
print(len(pos_clusters[0]))#channels
print(len(pos_clusters[0][0]))#clusters
print(len(pos_clusters[0][0][0]))#neuron indices for cluster

In [ ]:
print(len(neurons))#files
print(len(neurons[0]))#clusters
print(len(neurons[0][0]))#neuron indices of cluster
#print(len(neurons[0][0][0]))#neuron indices of cluster

In [ ]:
len(all_neurons[0])

In [ ]:
len(all_neurons[0][0])

In [ ]:
len(neurons[0][0])

In [ ]:
neurons[0][0]

In [ ]:
neurons[0][0]

In [ ]:
for file in neg_clusters:
    neurons_f=[]
    for channel in file:
        neurons_c=[]
        for neuron in channel:
            neurons_c.append(neuron)
        neurons_f.append(neurons_c)
    neurons.append(neurons_f)
len(neurons)

In [ ]:
len(neurons[0][3])

In [ ]:
adj_neur=[]
counter = 0
max_len=0

for file in neurons:
    #for clus in file:
    for neu in file:
        print('counter: ',counter,neu.shape[0])
        if neu.shape[0]>max_len:
            max_len=neu.shape[0]
        counter+=1
    #counter=0
print('max_len',max_len)

In [ ]:
for file in neurons:
    adj_neur_f=[]
    for neuron in file:
        if neuron.shape[0]<=max_len:
            diff = max_len-neuron.shape[0]
            adj_neur_f.append(np.concatenate((neuron,np.zeros([diff]))))
    adj_neur.append(adj_neur_f)
#print(max_len)
print(len(adj_neur),len(adj_neur[0]))

# Export (after sorting)

In [ ]:
import os
for i,file in enumerate(list_dir):
    name_data=os.path.basename(list_dir[i])
    save_data = 'After'+name_data
    #np.savetxt(r"C:\Data after SS\%save_data.txt" % save_data,adj_neur[i], delimiter=', ', fmt='%12.8f')
    np.savetxt("/Users/Gaia_1/Desktop/tesi/Data after SS/%s.txt" % save_data,adj_neur[i], delimiter=', ', fmt='%12.8f')
    

# Point Process

In [ ]:
def Bayesian_mixture_model(ISI_data):
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))
        #w = pm.Dirichlet('w', a=np.array([1., .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)
        #like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2], observed=ISI_data)

        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)
        
        #ppc_trace = pm.sample_posterior_predictive(trace,model=model)
        
    map_estimate = pm.find_MAP(model=model)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]

    del map_estimate['w']


    return map_estimate#, ppc_trace


In [ ]:
list_dir=glob.glob("Data after SS/*.txt")

In [ ]:
list_dir_ok = []
for net in list_dir:
    #if 'After' not in net:
        print(net,'ok')
        list_dir_ok.append(net)

In [ ]:
dataframe = pd.DataFrame()
counter_net=1
counter=0
target = 0
for net in list_dir_ok:
    print(counter_net,') ',net)
    counter_net+=1
    list_neurons = np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in list_neurons:
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>800:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'Healthy' in net:
            print('target Healthy')
            map_estimate['Target']=0
            
        elif 'healthy' in net:
            print('target healthy')
            map_estimate['Target']=0
        elif 'health' in net:
            print('target health')
            map_estimate['Target']=0

        else:
            print('target pathological')
            map_estimate['Target']=1
        
        if 'KA' in net:
            map_estimate['Stimulation']=1
            
        elif 'stimulation' in net:
            map_estimate['Stimulation']=1
        else:
            map_estimate['Stimulation']=0
            
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)

In [ ]:
final = dataframe.T
final.to_csv('Data after PP/DataAfter')

# Classification

In [ ]:
dataset = pd.read_csv('Data after PP/DataAfter')

In [ ]:
dataset = dataset.drop(['Unnamed: 0'],axis=1)

In [ ]:
dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = dataset.drop(['Target'],axis=1)
y= dataset['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(4)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3,svd_solver='full')
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.fit(X_train).explained_variance_ratio_)

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
params = {'max_depth':np.linspace(2,10,dtype='int'),'criterion':['gini','entropy']}

clf = DecisionTreeClassifier(random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

gs_pca = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)
gs_pca = gs_pca.fit(X_train_pca,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
#roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
#print('ROC AUC: ',roc_auc)
print('\n\n Result with PCA\n')
print("Best parameters set found :",gs_pca.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs_pca.predict(X_test_pca)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
#roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
#print('ROC AUC: ',roc_auc)

Logistic Regression

In [ ]:
params = {'C':np.logspace(-3,3,50),'penalty':['l1','l2','elasticnet']}

clf = LogisticRegression(solver='liblinear',random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

Random Forest

In [ ]:
params = {'n_estimators':np.linspace(10,150,dtype='int',num=10),'max_depth':np.linspace(2,5,dtype='int'),'criterion':['gini','entropy']}

clf = RandomForestClassifier(n_jobs=-1,random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

Neural Net

In [ ]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            20, activation="relu", input_shape=(X_train.shape[1],)
        ),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

In [ ]:
metrics = [
    'accuracy',
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)



history=model.fit(
    X_train,
    y_train,
    batch_size=10,
    epochs=200,
    validation_data=(X_test, y_test),

)

In [ ]:
# list all data in history
print(history.history.keys())
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_true, y_pred = y_test, model.predict_classes(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)